### Распределенное обучениеи с использованием PyTorch и Horovod для MNIST

In [1]:
import torch

Используйте Google Colab для запуска этого ноутбука - https://colab.research.google.com/

In [ ]:
%%writefile learn_hvd.py

import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

# Параметры обучения
batch_size = 100
num_epochs = 5
momentum = 0.5
log_interval = 100

def train_one_epoch(model, device, data_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(data_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(data_loader) * len(data),
                100. * batch_idx / len(data_loader), loss.item()))
            
from time import time
import os

LOG_DIR = os.path.join('./logs/', str(time()), 'MNISTDemo')
os.makedirs(LOG_DIR)

def save_checkpoint(model, optimizer, epoch):
  filepath = LOG_DIR + '/checkpoint-{epoch}.pth.tar'.format(epoch=epoch)
  state = {
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
  }
  torch.save(state, filepath)

import torch.optim as optim
from torchvision import datasets, transforms

def train(learning_rate):
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

  train_dataset = datasets.MNIST(
    'data', 
    train=True,
    download=True,
    transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]))
  data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

  model = Net().to(device)

  optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)

  for epoch in range(1, num_epochs + 1):
    train_one_epoch(model, device, data_loader, optimizer, epoch)
    save_checkpoint(model, optimizer, epoch)

import horovod.torch as hvd

def train_hvd(learning_rate):
  hvd.init()  # Иницииализация
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  
  if device.type == 'cuda':
    torch.cuda.set_device(hvd.local_rank())

  train_dataset = datasets.MNIST(
    root='data-%d'% hvd.rank(),  # каждый обработчики в своей папке
    train=True, 
    download=True,
    transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
  )

  from torch.utils.data.distributed import DistributedSampler
  
  train_sampler = DistributedSampler(train_dataset, num_replicas=hvd.size(), rank=hvd.rank())
  train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler)

  model = Net().to(device)
  
  optimizer = optim.SGD(model.parameters(), lr=learning_rate * hvd.size(), momentum=momentum)

  # оборачиваем оптимизатор в Horovod DistributedOptimizer
  optimizer = hvd.DistributedOptimizer(optimizer, named_parameters=model.named_parameters())
  
  # Ставим для всех моделей начальные параметры одинаковыми
  hvd.broadcast_parameters(model.state_dict(), root_rank=0)

  for epoch in range(1, num_epochs + 1):
    train_one_epoch(model, device, train_loader, optimizer, epoch)
    # Сохраняем только в одном обработчике
    if hvd.rank() == 0:
      save_checkpoint(model, optimizer, epoch)
  

if __name__ == '__main__':
  train_hvd(0.001)


Overwriting learn_hvd.py


In [ ]:
from learn_hvd import *

### Обучение MNIST

In [ ]:
train(learning_rate = 0.001)

/content/learn_hvd.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.361788
Train Epoch: 1 [10000/60000 (17%)]	Loss: 2.274706
Train Epoch: 1 [20000/60000 (33%)]	Loss: 2.295054
Train Epoch: 1 [30000/60000 (50%)]	Loss: 2.246834
Train Epoch: 1 [40000/60000 (67%)]	Loss: 2.238976
Train Epoch: 1 [50000/60000 (83%)]	Loss: 2.138915
Train Epoch: 2 [0/60000 (0%)]	Loss: 2.160299
Train Epoch: 2 [10000/60000 (17%)]	Loss: 2.056116
Train Epoch: 2 [20000/60000 (33%)]	Loss: 1.922915
Train Epoch: 2 [30000/60000 (50%)]	Loss: 1.867282
Train Epoch: 2 [40000/60000 (67%)]	Loss: 1.603615
Train Epoch: 2 [50000/60000 (83%)]	Loss: 1.694041
Train Epoch: 3 [0/60000 (0%)]	Loss: 1.558566
Train Epoch: 3 [10000/60000 (17%)]	Loss: 1.332901
Train Epoch: 3 [20000/60000 (33%)]	Loss: 1.320291
Train Epoch: 3 [30000/60000 (50%)]	Loss: 1.354439
Train Epoch: 3 [40000/60000 (67%)]	Loss: 1.175195
Train Epoch: 3 [50000/60000 (83%)]	Loss: 1.178067
Train Epoch: 4 [0/60000 (0%)]	Loss: 1.099002
Train Epoch: 4 [10000/60000 (17%)]	Loss: 1.088101
Train Epoch: 4 [2000

### Horovod

In [ ]:
! pip3 install horovod[all-frameworks]

In [ ]:
! horovodrun 

usage: horovodrun [-h] [-v] -np NP [-cb] [--disable-cache]
                  [--start-timeout START_TIMEOUT] [--network-interface NICS]
                  [--output-filename OUTPUT_FILENAME] [--verbose]
                  [--config-file CONFIG_FILE] [-p SSH_PORT]
                  [-i SSH_IDENTITY_FILE]
                  [--fusion-threshold-mb FUSION_THRESHOLD_MB]
                  [--cycle-time-ms CYCLE_TIME_MS]
                  [--cache-capacity CACHE_CAPACITY]
                  [--hierarchical-allreduce | --no-hierarchical-allreduce]
                  [--hierarchical-allgather | --no-hierarchical-allgather]
                  [--autotune] [--autotune-log-file AUTOTUNE_LOG_FILE]
                  [--autotune-warmup-samples AUTOTUNE_WARMUP_SAMPLES]
                  [--autotune-steps-per-sample AUTOTUNE_STEPS_PER_SAMPLE]
                  [--autotune-bayes-opt-max-samples AUTOTUNE_BAYES_OPT_MAX_SAMPLES]
                  [--autotune-gaussian-process-noise AUTOTUNE_GAUSSIAN_PROCESS_NOISE

In [ ]:
! horovodrun -np 1 -H localhost:1 python3 learn_hvd.py

2020-11-02 14:29:12.286241: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
[1,0]<stdout>:Train Epoch: 1 [0/60000 (0%)]	Loss: 2.337323
[1,0]<stdout>:Train Epoch: 1 [10000/60000 (17%)]	Loss: 2.311920
[1,0]<stdout>:Train Epoch: 1 [20000/60000 (33%)]	Loss: 2.293706
[1,0]<stdout>:Train Epoch: 1 [30000/60000 (50%)]	Loss: 2.302980
[1,0]<stdout>:Train Epoch: 1 [40000/60000 (67%)]	Loss: 2.274255
[1,0]<stdout>:Train Epoch: 1 [50000/60000 (83%)]	Loss: 2.266550
[1,0]<stdout>:Train Epoch: 2 [0/60000 (0%)]	Loss: 2.245056
[1,0]<stdout>:Train Epoch: 2 [10000/60000 (17%)]	Loss: 2.221961
[1,0]<stdout>:Train Epoch: 2 [20000/60000 (33%)]	Loss: 2.153626
[1,0]<stdout>:Train Epoch: 2 [30000/60000 (50%)]	Loss: 2.129339
^C
